# Implementing PCA from Scratch

Table of Content:
- Dataset
- Implementation of PCA
- PCA without standardization
- PCA with standardization
- PCA with Sklearn


In the previous post, I talked about one of the most known and widely used methods, called Principal Component Analysis. 

It employs an efficient linear transformation, which **reduces the dimensionality of a high dimensional dataset while capturing the maximum information content**. 


A better way is to create a class, which is effective when you want to encapsulate data structures and procedures in one place. Moreover, it’s really easier to modify since you have all the code in this unique class.

# Dataset
Before implementing the PCA algorithm, we are going to import the breast cancer Wisconsin dataset, which contains the data regarding
the breast cancer diagnosed in 569 patients

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
import pandas as pd
import numpy as np
import random
from sklearn.datasets import load_breast_cancer
import plotly.express as px
data = load_breast_cancer(as_frame=True)
X,y,df_bre = data.data,data.target,data.frame
diz_target = {0:'malignant',1:'benign'}
y = np.array([diz_target[y1] for y1 in y])
df_bre['target'] = df_bre['target'].apply(lambda x: diz_target[x])
df_bre.shape
df_bre.head()

(569, 31)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,malignant
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,malignant
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,malignant
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,malignant
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,malignant


We can notice that there are 30 numerical features and a target variable, that specify if the tumour is benign (target=1) or malignant (target=0). I convert the target variable to a string since it’s not used by PCA and we only need it in the visualizations later.

In this case, we want to understand the difference in variability of the features when the tumour is benign or malignant. This is really hard to show it with a simple exploratory analysis since we have more than two covariates. For example, we can try to visualize a scatter matrix with only six features, coloured by the target variable.

In [5]:
fig = px.scatter_matrix(df_bre,dimensions=list(df_bre.columns)[:5], color="target")
fig.show()

Certainly, we can observe two different clusters in all these scatter plots, but it’s messy if we plot all the features at the same time. 

Consequently, we need a compact representation of this multivariate dataset, which can be provided by the Principal Component Analysis.

# Implementation of PCA
![](https://miro.medium.com/max/875/1*d52CQw0ETYnJcRmdw1pwcA.png)


The steps to obtain the principal components (or k dimensional feature vectors) are summarized in the illustration above. The same logic will be applied to build the class.


We define the PCA_impl class, which has three attributes initialized at the beginning. The most important attribute is the number of components we want to extract. Moreover, we can also reproduce the same results every time by setting random_state equal to True and standardizing the dataset only if we need it.

In [6]:
class PCA_impl:
    def __init__(self,n_components,random_state=None,standardize=True):
        self.n_components = n_components
        self.random_state = random_state
        self.standardize = standardize

    def fit(self,X):

        self.X_copy = X.copy().astype('float32')

        if self.standardize==True:
            self.mean_ = np.mean(self.X_copy, axis=0)
            self.std_ = np.std(self.X_copy, axis=0)
            self.X_copy -= self.mean_
            self.X_copy /= self.std_

        if self.random_state ==True:
            random.seed(0)

        cov_mat = np.cov(self.X_copy.T)    
        eigen_values, eigen_vectors = np.linalg.eig(cov_mat)
        sorted_index = np.argsort(eigen_values)[::-1]
        self.sorted_eigenvalue_ = eigen_values[sorted_index]
        self.sorted_eigenvectors_ = eigen_vectors[:,sorted_index]
        self.explained_variance_ = self.sorted_eigenvalue_[0:self.n_components]
        total_var = self.sorted_eigenvalue_.sum()
        self.explained_variance_ratio_ = self.explained_variance_ / total_var
        self.cum_var_explained = np.cumsum(self.explained_variance_ratio_)

        self.projection_matrix_ = self.sorted_eigenvectors_[:,0:self.n_components]
        
        return

    def fit_transform(self,X,y):
        self.fit(X)
        self.components_ = np.dot(self.X_copy,self.projection_matrix_)
        self.df = pd.DataFrame(data=self.components_, columns=['PC{}'.format(i+1) for i in range(self.n_components)])
        self.df['label'] = y
        return self.components_

    def pca_plot2d(self):
        fig = px.scatter(self.components_, x=0, y=1, color=self.df.label,labels={'0': 'PC 1', '1': 'PC 2'})
        fig.show()

    def pca_plot3d(self):
        fig = px.scatter_3d(self.components_, x=0, y=1,z=2, color=self.df.label,labels={'0': 'PC 1', '1': 'PC 2'})
        fig.show()    

This class also includes two methods, `fit` and `fit_transform`, similarly to the `scikit-learn’s PCA`. While the first method provides most of the procedure to calculate the principal components, the fit_transform method also applies the transformation on the original feature matrix X. 

In addition to these two methods, I also wanted to visualize the principal components without specifying every time the functions of Plotly Express. It can be really useful to speed up the analysis of the latent variables generated by PCA.


# PCA without standardization
Finally, the PCA_impl class is defined. We only need to call the class and the corresponding methods without any effort.

In [7]:
pca1 = PCA_impl(n_components=4,random_state=True,standardize=False)
X_new = pca1.fit_transform(X,y)
print('4D feature subspace:\n',X_new)
print('\nVariance Explained Ratio:\n',pca1.explained_variance_ratio_)
print('\nCumulative Variance Explained:\n',pca1.cum_var_explained)

4D feature subspace:
 [[2260.01388639 -187.96030082   17.91295273  -77.2629491 ]
 [2368.99375578  121.58742466  -66.05996862  -50.68968454]
 [2095.66520144  145.11398617  -32.37519053  -64.351628  ]
 ...
 [1414.37305601  153.5107474   -41.10784342  -78.32284775]
 [2224.72942843  140.08646838  -50.40752361  -92.21185471]
 [ 328.34369582   17.31413276   -6.77640537  -66.00371815]]

Variance Explained Ratio:
 [9.82044672e-01 1.61764899e-02 1.55751071e-03 1.20931967e-04]

Cumulative Variance Explained:
 [0.98204467 0.99822116 0.99977867 0.9998996 ]


We can access `thevar_explained` and `cum_var_explained` attributes, that were calculated within the fit and fit_transform methods. It’s worth noticing that we capture 98% with just one component. Let’s also visualize the 2D and 3D scatterplots using the method defined previously:

In [8]:
pca1.pca_plot2d()

From the visualization, we can observe that two clusters emerge, one marked in blue representing the patients with malignant cancer and the other regarding benign cancer. Moreover, it seems that the blue cluster contains much more variability than the other cluster. In addition, we see a slight overlapping between the two groups.


In [9]:
pca1.pca_plot3d()

Now, we look at the 3D scatterplot with the first three components. It’s less clear than the previous scatterplot, but a similar behaviour emerges even in this plot. There are surely two distinct groups based on the target variable.

 New information is discovered looking at this three-dimensional representation: two patients with malignant cancer appear to have completely different values with respect to all the other patients. This aspect could be slightly noticed looking at the 2D plot or in the scatter matrix we displayed previously.


# PCA with standardization
Let’s replicate the same procedure of the previous section. We only add the standardization at the beginning to check if there are any differences in the results.

In [10]:
pca1 = PCA_impl(n_components=4,random_state=True,standardize=True)
X_new = pca1.fit_transform(X,y)
print('4D feature subspace:\n',X_new)
print('\nCumulative Variance Explained:\n',pca1.cum_var_explained)

4D feature subspace:
 [[ 9.19283635  1.94858213 -1.12316523  3.63373252]
 [ 2.38780131 -3.76817322 -0.52929273  1.11826561]
 [ 5.73389543 -1.0751748  -0.55174787  0.91208485]
 ...
 [ 1.25617813 -1.90229754  0.56272916 -2.08922473]
 [10.37479311  1.67200958 -1.87703069 -2.35602824]
 [-5.47524483 -0.67063785  1.49044303 -2.29915489]]

Cumulative Variance Explained:
 [0.44272028 0.63243211 0.72636374 0.79238507]


Differently from the previous case, we can notice that the range of values regarding the principal components is more restricted and 80% of the variance explained is captured with three components. 

In particular, the contribution of the first component passed from 0.99 to 0.44. This can be justified by the fact that all variables have the same units of measure and, consequently, the PCA is able to give equal weight to each feature.

In [11]:
pca1.pca_plot2d()

The 3D representation is easier to read and comprehend. Finally, we can conclude that the two groups of patients have different feature variability. Moreover, there are still the two data points that lie apart from the rest of the data.

In [12]:
pca1.pca_plot3d()

# PCA with Sklearn
At this point, we can apply the PCA implemented by Sklearn to compare it with my implementation. I should point out that there are some differences to take into account in this comparison. While my implementation of PCA is based on the covariance matrix, the scikit-learn’s PCA involves the centering of the input data and employs the Singular Value Decomposition to project the data to a lower-dimensional space.

Before we saw that standardization is a very important step before applying PCA. Since the mean is already subtracted from each feature’s column by Sklearn’s algorithm, we only need to divide each numerical variable by its own standard deviation.

In [16]:
from sklearn.decomposition import PCA


X_copy = X.copy().astype('float32')
X_copy /= np.std(X_copy, axis=0)

pca = PCA(n_components=4,random_state=123)
components = pca.fit_transform(X_copy)
df = pd.DataFrame(data=components, columns=['PC{}'.format(i) for i in range(components.shape[1])])
df['label'] = y

print('4D feature subspace:\n',components)
print('\nVariance Explained Ratio:\n',pca.explained_variance_ratio_)
print('\nCumulative Variance Explained:\n',np.cumsum(pca.explained_variance_ratio_))

4D feature subspace:
 [[ 9.192834    1.9485828  -1.1231651   3.6337442 ]
 [ 2.3878047  -3.7681704  -0.5292885   1.1182373 ]
 [ 5.7338967  -1.0751724  -0.55174947  0.9120678 ]
 ...
 [ 1.2561803  -1.9022976   0.56272817 -2.0892222 ]
 [10.374794    1.6720108  -1.8770347  -2.3560228 ]
 [-5.4752436  -0.6706358   1.4904416  -2.2991889 ]]

Variance Explained Ratio:
 [0.4427202  0.18971181 0.0939317  0.06602138]

Cumulative Variance Explained:
 [0.4427202  0.632432   0.72636366 0.79238504]


In [17]:
fig = px.scatter(components, x=0, y=1, color=df.label,labels={'0': 'PC 1', '1': 'PC 2'})
fig.show()

In [18]:
fig = px.scatter_3d(components, x=0, y=1,z=2, color=df.label,labels={'0': 'PC 1', '1': 'PC 2','2':'PC 3'})
fig.show()

In this case, my implementation and the sklearn’s PCA provided the same results, but it can happen that sometimes they are slightly different if you use a different dataset